In [1]:
import argilla as rg
from datasets import load_dataset

In [2]:
SEED = 42
# TRAIN_NUM_SAMPLES = 400
# TEST_NUM_SAMPLES = 100

In [3]:
train_ds_full = load_dataset("clarin-pl/kpwr-ner", split="train")

Found cached dataset kpwr-ner (/home/jjamnicki/.cache/huggingface/datasets/clarin-pl___kpwr-ner/default/0.0.0/001e3d471298007e8412e3a6ccc06bec000dec1bce0cf8e0ba7e5b7e105b1342)


In [4]:
def _contains_any_label(example, labels):
    return any(map(lambda label: label in labels, example["ner"]))

# index: label (BIO)
include_labels = {
    20: "B-nam_liv_person", 98: "I-nam_liv_person",
    27: "B-nam_loc_gpe_city", 105: "I-nam_loc_gpe_city",
    29: "B-nam_loc_gpe_country", 107: "I-nam_loc_gpe_country"
}

assert len(include_labels) % 2 == 0

labels_num = int(len(include_labels) / 2)

In [5]:
TRAIN_DS_NAME = f"unlabelled_active_learninig_dataset"

In [6]:
train_dataset = train_ds_full \
                    .filter(lambda record: _contains_any_label(record, include_labels.keys()))

Loading cached processed dataset at /home/jjamnicki/.cache/huggingface/datasets/clarin-pl___kpwr-ner/default/0.0.0/001e3d471298007e8412e3a6ccc06bec000dec1bce0cf8e0ba7e5b7e105b1342/cache-e7fee70cda2db228.arrow


In [7]:
len(train_dataset)

3456

In [8]:
record_sample = train_dataset[0]
for key, item in record_sample.items():
    print(f"{key}: {item}\n")

tokens: ['Roboty', 'mają', 'kilkanaście', 'lat', 'i', 'pochodzą', 'z', 'USA', ',', 'Wysokie', 'napięcie', 'jest', 'dużo', 'młodsze', ',', 'powstało', 'w', 'Niemczech', '.']

lemmas: ['robota', 'maić', 'kilkanaście', 'rok', 'i', 'pochodzić', 'z', 'USA', ',', 'wysoki', 'napięcie', 'być', 'dużo', 'młody', ',', 'powstać', 'w', 'Niemcy', '.']

orth: ['subst:pl:nom:f', 'fin:pl:ter:imperf', 'num:pl:acc:m3:rec', 'subst:pl:gen:m3', 'conj', 'fin:pl:ter:imperf', 'prep:gen:nwok', 'subst:pl:gen:n', 'interp', 'adj:sg:nom:n:pos', 'subst:sg:nom:n', 'fin:sg:ter:imperf', 'num:pl:nom:n:rec', 'adj:sg:nom:n:com', 'interp', 'praet:sg:n:perf', 'prep:loc:nwok', 'subst:pl:loc:n', 'interp']

ner: [73, 160, 160, 160, 160, 160, 160, 29, 160, 73, 151, 160, 160, 160, 160, 160, 160, 29, 160]



In [9]:
def map_idx_to_label(example, labels_dict):
    # "O" if not in labels to be included
    return [
        labels_dict[idx] if idx in labels_dict.keys() else "O"
        for idx in example["ner"]
    ]

def datasets_to_rg(dataset):
    rg_records = [
        rg.TokenClassificationRecord(
            tokens=example["tokens"]
        )
        for example in dataset
    ]
    return rg.DatasetForTokenClassification(rg_records)

In [10]:
train_dataset_sample = train_dataset.shuffle(seed=SEED)#.select(range(TRAIN_NUM_SAMPLES))

Loading cached shuffled indices for dataset at /home/jjamnicki/.cache/huggingface/datasets/clarin-pl___kpwr-ner/default/0.0.0/001e3d471298007e8412e3a6ccc06bec000dec1bce0cf8e0ba7e5b7e105b1342/cache-fd256080dda81348.arrow


In [11]:
rg_records_train = datasets_to_rg(train_dataset_sample)

In [12]:
rg_dataset_train = rg.DatasetForTokenClassification(rg_records_train)

In [13]:
rg.log(rg_dataset_train, name=TRAIN_DS_NAME)

  0%|          | 0/3456 [00:00<?, ?it/s]

An open stream object is being garbage collected; call "stream.close()" explicitly.
An open stream object is being garbage collected; call "stream.close()" explicitly.
An open stream object is being garbage collected; call "stream.close()" explicitly.
An open stream object is being garbage collected; call "stream.close()" explicitly.
An open stream object is being garbage collected; call "stream.close()" explicitly.
An open stream object is being garbage collected; call "stream.close()" explicitly.
An open stream object is being garbage collected; call "stream.close()" explicitly.


3456 records logged to http://localhost:6900/datasets/argilla/unlabelled_active_learninig_dataset


BulkResponse(dataset='unlabelled_active_learninig_dataset', processed=3456, failed=0)

## dobry przykład do displacy ner

In [14]:
for record in rg_dataset_train:
    if "Hadze" in record.text:
        print(record.text)
        print(record.annotation)

Prezydent Sudanu Omar al - Baszir już zapowiedział , że jego kraj nie będzie współpracował z trybunałem w Hadze .
None
Robin Haase ( ur . 6 kwietnia 1987 roku w Hadze ) – holenderski tenisista , reprezentant w Pucharze Davisa .
None


In [15]:
# !jupyter nbconvert --to pdf kpwr-argilla-log.ipynb --output ./misc/kpwr-argilla-log.pdf